In [1]:
%cd ..

/home/zain/projects/matchmaker


In [2]:
import pandas as pd
from matchmaker.data import Interactions

import cudf
import cugraph

In [3]:
print("Importing data...")
df = pd.read_csv("examples/data/swipes.csv")

Importing data...


In [4]:
# load pandas dataframe into a GPU dataframe
gdf = cudf.DataFrame.from_pandas(df)

In [9]:
G = cugraph.Graph(directed=True)

# Pick 'like' as edge weight
G.from_cudf_edgelist(
    gdf,
    source="decidermemberid",
    destination="othermemberid",
    edge_attr="like",  # only one column allowed
    store_transposed=True)

In [10]:
# Compute PageRank on GPU
pagerank_df = cugraph.pagerank(G, alpha=0.85)

In [11]:
# Merge back node attributes
decider_nodes = gdf[["decidermemberid", "decidergender", "deciderdobyear", "decidersignuptimestamp"]] \
    .rename(columns={"decidermemberid": "user_id"})
other_nodes = gdf[["othermemberid", "othergender", "otherdobyear", "othersignuptimestamp"]] \
    .rename(columns={"othermemberid": "user_id"})

nodes = cudf.concat([decider_nodes, other_nodes]).drop_duplicates(subset=["user_id"])

# Merge pagerank
nodes = nodes.merge(pagerank_df, left_on="user_id", right_on="vertex", how="left")

In [12]:
nodes

,user_id,decidergender,deciderdobyear,decidersignuptimestamp,othergender,otherdobyear,othersignuptimestamp,vertex,pagerank
0,2182822,F,1994,2020-02-19 14:31:08,<NA>,<NA>,<NA>,2182822,0.000018
1,482872,M,1988,2018-05-20 13:06:21,<NA>,<NA>,<NA>,482872,0.000004
2,753587,M,1986,2018-10-14 16:11:48,<NA>,<NA>,<NA>,753587,0.000004
3,3681841,M,1993,2020-11-16 19:07:45,<NA>,<NA>,<NA>,3681841,0.000004
4,1005078,M,1996,2019-01-15 22:26:12,<NA>,<NA>,<NA>,1005078,0.000004
...,...,...,...,...,...,...,...,...,...
171103,2146887,<NA>,<NA>,<NA>,M,2001,2020-02-11 22:56:29,2146887,0.000004
171104,912226,<NA>,<NA>,<NA>,M,1996,2018-12-11 02:09:10,912226,0.000004
171105,1412760,<NA>,<NA>,<NA>,M,2001,2019-06-10 22:32:31,1412760,0.000004
171106,720870,<NA>,<NA>,<NA>,M,1995,2018-10-01 14:03:15,720870,0.000004
